# Forte Tutorial 6: Orbital localization

---

In this tutorial we are going to explore how to localize orbitals and visualize them in Jupyter notebooks using the Python API.

Let's import the `psi4` and `forte` modules, including `the forte.utils` submodule

In [ ]:
import psi4
import forte
import forte.utils

We will start by generating SCF orbitals for methane via psi4 using the `forte.util.psi4_scf` function

In [ ]:
xyz = """
0 1
C       -1.9565506735      0.4146729724      0.0000000000                 
H       -0.8865506735      0.4146729724      0.0000000000                 
H       -2.3132134555      1.1088535618     -0.7319870007                 
H       -2.3132183114      0.7015020975      0.9671697106                 
H       -2.3132196063     -0.5663349614     -0.2351822830
symmetry c1"""

E_scf, wfn = forte.utils.psi4_scf(xyz, 'sto-3g', 'rhf', functional = 'hf')

Next we start forte, setup the MOSpaceInfo object specifying the number of orbitals, and start the integral object

In [ ]:
from forte import forte_options

forte.startup()

options = forte.forte_options

mos_spaces = {'RESTRICTED_DOCC' : [5], 'ACTIVE' : [0]}
mo_space_info = forte.make_mo_space_info_from_map(wfn,mos_spaces,[])
scf_info = forte.SCFInfo(wfn)
ints = forte.make_forte_integrals(wfn, options, mo_space_info)

## Localizing the orbitals
To localize the orbitals we create a `Localize` object. We have to passa few object to perform the localization

In [ ]:
localizer = forte.Localize(forte.forte_options, ints, mo_space_info)

Once the `Localize` object is created, we specify which we want to be localized and can compute the unitary transformation

In [ ]:
localizer.set_orbital_space(['RESTRICTED_DOCC'])
localizer.compute_transformation()

From the localizer we can then extract the unitary transformation matrix that correspond to the orbital localizaition. Here we get the alpha part

In [ ]:
Ua = localizer.get_Ua()

We are now ready to read the MOs from psi4 and transform them by computing the product $\mathbf{C}' = \mathbf{C} \mathbf{U}$. We then place the orbitals back into psi4 by calling the `copy` function on `wfn.Ca()`. We have to do this because this function returns a smart pointer to the matrix that holds $\mathbf{C}$. If we assigned `Ca_local` via `wfn.Ca() = Ca_local` we would not change the orbitals in psi4.

In [ ]:
Ca = wfn.Ca()
Ca_local = psi4.core.doublet(Ca,Ua,False,False)
wfn.Ca().copy(Ca_local)

Lastly we generate cube files for all the occupied orbitals and visualize them. The resulting orbitals consist of a core orbital (which cannot be seen) and four localized C-H $\sigma$ bond orbitals

In [ ]:
cubes = forte.utils.psi4_cubeprop(wfn,path='cubes',nocc=5,nvir=0, load=True)
plot = forte.utils.cube_viewer(cubes)
plot

In [ ]:
forte.cleanup()